In [1]:
using Pkg
using Revise
Pkg.activate(joinpath(@__DIR__, "../../"))
using Globtim
using DynamicPolynomials, DataFrames
# Constants and Parameters
const n, a, b = 3, 1, 1
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = Rastringin # Objective function

  Activating project at `~/globtim`


Rastringin (generic function with 1 method)

We set the number of samples used to generate the approximant. It is annoying that the error goes up while the degree has increased.

In [2]:
rand_center = [0.0, 0.0, 0,];
d = 10 # initial degree 
SMPL = 40 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor, 
                degree_max =d+4
                )
pol_cheb = Constructor(TR, d, basis=:chebyshev)
pol_lege = Constructor(TR, d, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

current L2-norm: 8.892146854778785e-5
current L2-norm: 0.06280960975634996


Solve the system of partial derivatives using `Msolve`. 

In [3]:
# df_cheb = solve_and_parse(pol_cheb, x, f, TR)
pts_cheb = solve_polynomial_system(x, TR.dim, d, pol_cheb.coeffs; basis=:chebyshev)
df_cheb = process_critical_points(pts_cheb, f, TR)
sort!(df_cheb, :z, rev=false)

Tracking 729 paths...   0%|▏                            |  ETA: 0:24:40



Tracking 729 paths...  71%|████████████████████▌        |  ETA: 0:00:02



Tracking 729 paths... 100%|█████████████████████████████| Time: 0:00:04
  # paths tracked:                  729
  # non-singular solutions (real):  729 (125)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         729 (125)

=== Starting Critical Points Processing (dimension: 3) ===
Processed 125 points (0.037s)


Row,x1,x2,x3,z
,Float64,Float64,Float64,Float64
1,6.85121e-19,-3.07804e-16,-4.30319e-17,-30.0
2,-9.24499e-18,0.991184,1.03506e-16,-29.0022
3,-8.41347e-18,-0.991184,3.05601e-16,-29.0022
4,-0.991184,-2.66274e-16,-1.16693e-16,-29.0022
5,-1.06544e-17,-3.46719e-16,0.991184,-29.0022
6,0.991184,-2.70042e-16,-1.30726e-16,-29.0022
7,-4.85657e-18,-3.72312e-16,-0.991184,-29.0022
8,-0.991184,-3.24631e-16,0.991184,-28.0044
9,0.991184,0.991184,-2.83739e-17,-28.0044


In [4]:
using StaticArrays
using GLMakie
include("LevelSetViz.jl")
using .LevelSetViz

In [5]:
grid = scale_factor * generate_grid(3, 100)  # 3D grid
values = map(f, grid); # Prepare level set data for specific level

In [7]:
fig = create_level_set_visualization(f, grid, df_cheb, (-30.0, 30.))